In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('instance').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/18 21:44:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
orders = spark.read.csv('/Users/adhoc/git/retail_db/orders', schema='order_customer_id long, order_date date, order_id long, order_status string')
orders.printSchema()
orders.show()

root
 |-- order_customer_id: long (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_id: long (nullable = true)
 |-- order_status: string (nullable = true)

+-----------------+----------+--------+---------------+
|order_customer_id|order_date|order_id|   order_status|
+-----------------+----------+--------+---------------+
|                1|2013-07-25|   11599|         CLOSED|
|                2|2013-07-25|     256|PENDING_PAYMENT|
|                3|2013-07-25|   12111|       COMPLETE|
|                4|2013-07-25|    8827|         CLOSED|
|                5|2013-07-25|   11318|       COMPLETE|
|                6|2013-07-25|    7130|       COMPLETE|
|                7|2013-07-25|    4530|       COMPLETE|
|                8|2013-07-25|    2911|     PROCESSING|
|                9|2013-07-25|    5657|PENDING_PAYMENT|
|               10|2013-07-25|    5648|PENDING_PAYMENT|
|               11|2013-07-25|     918| PAYMENT_REVIEW|
|               12|2013-07-25|    1837|  

In [37]:
order_items = spark.read.csv('/Users/adhoc/git/retail_db/order_items', schema='order_item_id long, order_item_order_id long, order_item_product_id long, order_item_quantity double, order_item_subtotal double, order_item_product_price double')
order_items.printSchema()
order_items.show()

root
 |-- order_item_id: long (nullable = true)
 |-- order_item_order_id: long (nullable = true)
 |-- order_item_product_id: long (nullable = true)
 |-- order_item_quantity: double (nullable = true)
 |-- order_item_subtotal: double (nullable = true)
 |-- order_item_product_price: double (nullable = true)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                1.0|             299.98|                  299.98|
|            2|                  2|                 1073|                1.0|             199.99|                  199.99|
|            3|                  2|                  502|                5.0| 

## Common aggreagate functions available as part of pyspark.sql.functions
* `count`
* `sum`
*  `min`
*  `max`
*  `avg`

Also statistical functions such as `stddev`

In [17]:
orders.select(count('*')).show()

from pyspark.sql.functions import count
orders.count()

+--------+
|count(1)|
+--------+
|   68883|
+--------+



68883

In [18]:
orders.groupBy('order_status').agg(count('*')).show()

+---------------+--------+
|   order_status|count(1)|
+---------------+--------+
|PENDING_PAYMENT|   15030|
|       COMPLETE|   22899|
|        ON_HOLD|    3798|
| PAYMENT_REVIEW|     729|
|     PROCESSING|    8275|
|         CLOSED|    7556|
|SUSPECTED_FRAUD|    1558|
|        PENDING|    7610|
|       CANCELED|    1428|
+---------------+--------+



## Total aggregations on Spark Data Frame
Get number of items, total quantity as well as revenue for a given order item order id eg. 2

In [53]:
from pyspark.sql.functions import count, sum

order_items.filter('order_item_order_id = 2').show(),
order_items.filter('order_item_order_id = 2').select(sum('order_item_subtotal').alias('order_revenue')).show(),
order_items.filter('order_item_order_id = 2').select(sum('order_item_quantity').alias('order total quantity')).show()
order_items.count()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            2|                  2|                 1073|                1.0|             199.99|                  199.99|
|            3|                  2|                  502|                5.0|              250.0|                    50.0|
|            4|                  2|                  403|                1.0|             129.99|                  129.99|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+

+-------------+
|order_revenue|
+-------------+
|       579.98|
+-------------+

+--------------------+
|order total quantity|
+----------

172198

In [47]:
order_items. \
    filter('order_item_order_id = 2'). \
    select(count('order_item_order_id').alias('order_item_count'),
           sum('order_item_quantity').alias('order_quantity'),
           sum('order_item_subtotal').alias('order_revenue'),). \
    show()

+----------------+--------------+-------------+
|order_item_count|order_quantity|order_revenue|
+----------------+--------------+-------------+
|               3|           7.0|       579.98|
+----------------+--------------+-------------+



## Getting count of the Spark Data Frame

In [48]:
order_items.count() # action - execution is called immediately

172198

In [49]:
from pyspark.sql.functions import count
order_items.select(count('*')).show()

+--------+
|count(1)|
+--------+
|  172198|
+--------+



## Overview `groupBy` on Spark Data Frame

In [55]:
# min on every column where there is no column specified in groupBy 
order_items.groupBy().min().show()

+------------------+------------------------+--------------------------+------------------------+------------------------+-----------------------------+
|min(order_item_id)|min(order_item_order_id)|min(order_item_product_id)|min(order_item_quantity)|min(order_item_subtotal)|min(order_item_product_price)|
+------------------+------------------------+--------------------------+------------------------+------------------------+-----------------------------+
|                 1|                       1|                        19|                     1.0|                    9.99|                         9.99|
+------------------+------------------------+--------------------------+------------------------+------------------------+-----------------------------+



In [60]:
orders.groupBy().min().show() # mins on numeric types only - weird
orders.dtypes

+----------------------+-------------+
|min(order_customer_id)|min(order_id)|
+----------------------+-------------+
|                     1|            1|
+----------------------+-------------+



[('order_customer_id', 'bigint'),
 ('order_date', 'date'),
 ('order_id', 'bigint'),
 ('order_status', 'string')]

## Perform grouped aggragations using direct functions on Spark Data Frame

In [62]:
order_items_grouped = order_items.groupBy('order_item_order_id')
type(order_items_grouped)

pyspark.sql.group.GroupedData

In [68]:
from pyspark.sql.functions import count, sum
order_items_grouped.count().show()

+-------------------+-----+
|order_item_order_id|count|
+-------------------+-----+
|                 29|    5|
|                474|    5|
|                964|    4|
|               1677|    5|
|               1806|    3|
|               1950|    5|
|               2214|    2|
|               2250|    4|
|               2453|    4|
|               2509|    4|
|               2529|    1|
|               2927|    5|
|               3091|    3|
|               3764|    1|
|               4590|    5|
|               4894|    4|
|               5385|    4|
|               5409|    3|
|               6721|    2|
|               7225|    4|
+-------------------+-----+
only showing top 20 rows



In [69]:
order_items_grouped. \
    count(). \
    withColumnRenamed('count', 'other_count'). \
    show()

+-------------------+-----------+
|order_item_order_id|other_count|
+-------------------+-----------+
|                 29|          5|
|                474|          5|
|                964|          4|
|               1677|          5|
|               1806|          3|
|               1950|          5|
|               2214|          2|
|               2250|          4|
|               2453|          4|
|               2509|          4|
|               2529|          1|
|               2927|          5|
|               3091|          3|
|               3764|          1|
|               4590|          5|
|               4894|          4|
|               5385|          4|
|               5409|          3|
|               6721|          2|
|               7225|          4|
+-------------------+-----------+
only showing top 20 rows



In [71]:
# Get sum of all numeric fields
order_items_grouped.sum().show()

+-------------------+------------------+------------------------+--------------------------+------------------------+------------------------+-----------------------------+
|order_item_order_id|sum(order_item_id)|sum(order_item_order_id)|sum(order_item_product_id)|sum(order_item_quantity)|sum(order_item_subtotal)|sum(order_item_product_price)|
+-------------------+------------------+------------------------+--------------------------+------------------------+------------------------+-----------------------------+
|                 29|               425|                     145|                      3897|                     9.0|                 1109.85|            909.9300000000001|
|                474|              5815|                    2370|                      4508|                    13.0|       774.8199999999999|           374.94000000000005|
|                964|              9586|                    3856|                      2964|                    11.0|       739.8800000

In [73]:
orders.groupBy('order_date').sum().show()

+----------+----------------------+-------------+
|order_date|sum(order_customer_id)|sum(order_id)|
+----------+----------------------+-------------+
|2013-09-09|               3087782|      1499494|
|2013-09-19|               2830266|      1300695|
|2014-06-03|               7200647|       793696|
|2013-09-12|               2898570|      1125297|
|2014-01-24|               6326762|      1010050|
|2014-02-16|               9143813|      1559341|
|2014-06-11|               7447538|       879243|
|2013-11-18|               4059151|      1196066|
|2014-02-18|               7931814|      1413476|
|2013-08-14|               3371810|      1155162|
|2013-10-05|               3236782|      1265413|
|2014-07-04|               8742268|       895348|
|2014-07-06|               6222972|       677105|
|2013-09-18|               3841581|      1511642|
|2013-09-20|               2247287|       851705|
|2013-09-25|               4304871|      1778180|
|2014-06-13|              14029444|      1609465|


In [78]:
order_items_grouped = order_items. \
    select('order_item_order_id', 'order_item_quantity', 'order_item_subtotal'). \
    groupBy('order_item_order_id')

In [88]:
order_items_grouped.sum().show()
order_items_grouped.sum('order_item_quantity', 'order_item_subtotal').show()

from pyspark.sql.functions import round
order_items_grouped. \
    sum('order_item_quantity', 'order_item_subtotal'). \
    toDF('order_item_quantity', 'order_quantity', 'order_revenue'). \
    withColumn('order_revenue', round('order_revenue', 2)). \
    show()

+-------------------+------------------------+------------------------+------------------------+
|order_item_order_id|sum(order_item_order_id)|sum(order_item_quantity)|sum(order_item_subtotal)|
+-------------------+------------------------+------------------------+------------------------+
|                 29|                     145|                     9.0|                 1109.85|
|                474|                    2370|                    13.0|       774.8199999999999|
|                964|                    3856|                    11.0|       739.8800000000001|
|               1677|                    8385|                    14.0|       649.9200000000001|
|               1806|                    5418|                     8.0|                  789.94|
|               1950|                    9750|                    12.0|      1015.8700000000001|
|               2214|                    4428|                     5.0|                  449.96|
|               2250|         

## Grouped aggregations using `agg` on a Spark Data Frame

In [90]:
order_item_grouped = order_items. \
    groupBy('order_item_order_id')

In [94]:
order_items_grouped.sum('order_item_quantity', 'order_item_subtotal').show()

+-------------------+------------------------+------------------------+
|order_item_order_id|sum(order_item_quantity)|sum(order_item_subtotal)|
+-------------------+------------------------+------------------------+
|                 29|                     9.0|                 1109.85|
|                474|                    13.0|       774.8199999999999|
|                964|                    11.0|       739.8800000000001|
|               1677|                    14.0|       649.9200000000001|
|               1806|                     8.0|                  789.94|
|               1950|                    12.0|      1015.8700000000001|
|               2214|                     5.0|                  449.96|
|               2250|                    10.0|                  889.94|
|               2453|                     7.0|       999.9300000000001|
|               2509|                     4.0|                  889.94|
|               2529|                     1.0|                  

In [98]:
order_items_grouped.agg(sum('order_item_quantity'), sum('order_item_subtotal')).show()
order_items_grouped.agg(sum('order_item_quantity').alias('order_quantity'), round(sum('order_item_subtotal'), 2).alias('order_revenue')).show()

+-------------------+------------------------+------------------------+
|order_item_order_id|sum(order_item_quantity)|sum(order_item_subtotal)|
+-------------------+------------------------+------------------------+
|                 29|                     9.0|                 1109.85|
|                474|                    13.0|       774.8199999999999|
|                964|                    11.0|       739.8800000000001|
|               1677|                    14.0|       649.9200000000001|
|               1806|                     8.0|                  789.94|
|               1950|                    12.0|      1015.8700000000001|
|               2214|                     5.0|                  449.96|
|               2250|                    10.0|                  889.94|
|               2453|                     7.0|       999.9300000000001|
|               2509|                     4.0|                  889.94|
|               2529|                     1.0|                  

In [102]:
order_items_grouped.agg({'order_item_quantity': 'sum', 'order_item_subtotal': 'sum'}).show()
order_items_grouped. \
    agg({'order_item_quantity': 'sum', 'order_item_subtotal': 'sum'}). \
    toDF('order_id', 'order_revenue', 'order_quantity',). \
    withColumn('order_revenue', round('order_revenue', 2)). \
    show()

+-------------------+------------------------+------------------------+
|order_item_order_id|sum(order_item_subtotal)|sum(order_item_quantity)|
+-------------------+------------------------+------------------------+
|                 29|                 1109.85|                     9.0|
|                474|       774.8199999999999|                    13.0|
|                964|       739.8800000000001|                    11.0|
|               1677|       649.9200000000001|                    14.0|
|               1806|                  789.94|                     8.0|
|               1950|      1015.8700000000001|                    12.0|
|               2214|                  449.96|                     5.0|
|               2250|                  889.94|                    10.0|
|               2453|       999.9300000000001|                     7.0|
|               2509|                  889.94|                     4.0|
|               2529|                   59.99|                  

In [107]:
# keys are to be unique
order_items_grouped. \
    agg({'order_item_quantity': 'sum', 'order_item_quantity': 'min'}). \
    show()
# column approach is to be used in case we want to repeat the key
from pyspark.sql.functions import min
order_items_grouped. \
    agg(
        sum('order_item_quantity'), 
        min('order_item_quantity')
    ). \
    show()



+-------------------+------------------------+
|order_item_order_id|min(order_item_quantity)|
+-------------------+------------------------+
|                 29|                     1.0|
|                474|                     1.0|
|                964|                     1.0|
|               1677|                     1.0|
|               1806|                     1.0|
|               1950|                     1.0|
|               2214|                     1.0|
|               2250|                     1.0|
|               2453|                     1.0|
|               2509|                     1.0|
|               2529|                     1.0|
|               2927|                     1.0|
|               3091|                     1.0|
|               3764|                     2.0|
|               4590|                     1.0|
|               4894|                     1.0|
|               5385|                     1.0|
|               5409|                     1.0|
|            